In [ ]:
import sys
import os
import torch
import numpy as np
import torchvision
import cv2
import WDNet

basic_path = ''
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
torch.cuda.is_available()

In [ ]:
args ={}
args['epoch'] = 1
args['batch_size'] = 60 # 12 for 16Gb
args['save_dir'] = basic_path+'../model_WDNet'
args['result_dir'] = ''
args['dataset'] = basic_path+'../data_CLWD'
args['log_dir'] = basic_path+'../model_WDNet'
args['gpu_mode'] = True
args['input_size'] = 28 # не знаю что это
args['dataloader_workers'] = 28
args['gan_type'] = 'CGAN'
args['lrG'] = 0.0002
args['lrD'] = 0.0002
args['beta1'] = 0.5
args['beta2'] = 0.999
args

In [ ]:
wdnet = WDNet.WDNet(args)

In [ ]:
wdnet.train()